In [15]:
# reformat into jsonl file
import os
import json
os.mkdir('data_2017-09/webpages_tmp')
with open('data_2017-09/webpages_tmp/webpages_tmp.jsonl', 'w') as f:
    for i,file in enumerate(os.listdir('data_2017-09/webpages/')):
        webpage = json.load(open('data_2017-09/webpages/' + file, 'r'))
        f.write(json.dumps(webpage) + '\n')
        if i % 10000 == 0: print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [18]:
# sets up the pyserini sparse index
!python3 -m pyserini.index.lucene --stopwords "data_2017-09/stopwords.txt" --collection JsonCollection --input "data_2017-09/webpages_tmp" --index "data_2017-09/pyserini/pyserini_index" --generator DefaultLuceneDocumentGenerator --threads 10 -verbose

2022-05-06 11:52:03,425 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to DEBUG
2022-05-06 11:52:03,427 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-05-06 11:52:03,427 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-05-06 11:52:03,428 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: data_2017-09/webpages_tmp
2022-05-06 11:52:03,428 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-05-06 11:52:03,428 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-05-06 11:52:03,428 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 10
2022-05-06 11:52:03,429 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-05-06 11:52:03,429 INFO  [main] index.IndexCollec

In [19]:
os.remove('data_2017-09/webpages_tmp/webpages_tmp.jsonl')
os.rmdir('data_2017-09/webpages_tmp')

In [21]:
!python3 -m pyserini.search.lucene --batch-size 50 --threads 10 --max-passage-hits 10 --index "data_2017-09/pyserini/pyserini_index" --topics "data_2017-09/queries/queries_val.tsv" --output out/bm25_runs/run.val_3_0.9.txt --bm25 --k1 3 --b 0.9

# note MAP = MRR when there is exactly one relevant result
# https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank
!python3 -m pyserini.eval.trec_eval -m P.1 "data_2017-09/queries/relevance_scores.txt" out/bm25_runs/run.val_3_0.9.txt

Setting BM25 parameters: k1=3.0, b=0.9
Running data_2017-09/queries/queries_val.tsv topics, saving to out/bm25_runs/run.val.txt...
100%|█████████████████████████████████████| 15464/15464 [10:01<00:00, 25.71it/s]
/home/kjros2/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/kjros2/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-m', 'P.1', 'data_2017-09/queries/relevance_scores.txt', 'out/bm25_runs/run.val.txt']
Results:
P_1                   	all	0.1672

